# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,21.07,76,0,0.45,US,1725501252
1,1,tiksi,71.6872,128.8694,2.16,99,100,5.13,RU,1725501252
2,2,atafu village,-8.5421,-172.5159,27.83,79,74,6.84,TK,1725501252
3,3,tolanaro,-25.0319,46.9987,20.04,88,40,4.12,MG,1725501253
4,4,constantia,44.1833,28.6500,17.61,65,6,5.51,RO,1725501253


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color = "City",
    hover_cols=["City", "Humidity"]
)

# Display the map
city_data_map



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
Filtered_cities = city_data_df[(city_data_df["Humidity"] > 50) & (city_data_df["Max Temp"] < 25)]

# Drop any rows with null values
filtered_cities = Filtered_cities.dropna()

# Display sample data
filtered_cities.head(25)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fortuna,40.5982,-124.1573,21.07,76,0,0.45,US,1725501252
1,1,tiksi,71.6872,128.8694,2.16,99,100,5.13,RU,1725501252
3,3,tolanaro,-25.0319,46.9987,20.04,88,40,4.12,MG,1725501253
4,4,constantia,44.1833,28.6500,17.61,65,6,5.51,RO,1725501253
5,5,marietta,33.9526,-84.5499,24.23,60,75,4.12,US,1725501253
7,7,puerto natales,-51.7236,-72.4875,3.66,78,34,2.87,CL,1725501253
8,8,talnakh,69.4865,88.3972,4.10,67,100,3.42,RU,1725501253
9,9,waitangi,-43.9535,-176.5597,11.01,73,6,1.34,NZ,1725501253
10,10,gadzhiyevo,69.2551,33.3362,15.51,64,100,8.40,RU,1725501253
11,11,namibe,-15.1961,12.1522,17.47,89,0,1.61,AO,1725501253


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(25)

,City,Country,Lat,Lng,Humidity,Hotel Name
0,fortuna,US,40.5982,-124.1573,76,
1,tiksi,RU,71.6872,128.8694,99,
3,tolanaro,MG,-25.0319,46.9987,88,
4,constantia,RO,44.1833,28.6500,65,
5,marietta,US,33.9526,-84.5499,60,
7,puerto natales,CL,-51.7236,-72.4875,78,
8,talnakh,RU,69.4865,88.3972,67,
9,waitangi,NZ,-43.9535,-176.5597,73,
10,gadzhiyevo,RU,69.2551,33.3362,64,
11,namibe,AO,-15.1961,12.1522,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")
print (geoapify_key)
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    print ("response=", response)
    
    # Convert the API response to JSON format
    name_address = response.json()
    print(name_address)
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
5ca2fd4f7d6b4be5863bd570ed748faf
response= <Response [200]>
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Comfort Inn & Suites Redwood Country', 'ref': 'CA688', 'country': 'United States', 'country_code': 'us', 'state': 'California', 'county': 'Humboldt County', 'city': 'Fortuna', 'postcode': '95540', 'street': 'Riverwalk Drive', 'housenumber': '1583', 'lon': -124.15440942340052, 'lat': 40.578712100000004, 'state_code': 'CA', 'formatted': 'Comfort Inn & Suites Redwood Country, 1583 Riverwalk Drive, Fortuna, CA 95540, United States of America', 'address_line1': 'Comfort Inn & Suites Redwood Country', 'address_line2': '1583 Riverwalk Drive, Fortuna, CA 95540, United States of America', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details', 'details.contact'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Op

,City,Country,Lat,Lng,Humidity,Hotel Name
0,fortuna,US,40.5982,-124.1573,76,Comfort Inn & Suites Redwood Country
1,tiksi,RU,71.6872,128.8694,99,Арктика
3,tolanaro,MG,-25.0319,46.9987,88,Hôtel Mahavokey
4,constantia,RO,44.1833,28.6500,65,Hotel Dali
5,marietta,US,33.9526,-84.5499,60,Hilton Atlanta / Marietta Hotel & Conference C...
...,...,...,...,...,...,...
564,mahebourg,MU,-20.4081,57.7000,78,Grand Bel Air
566,buynaksk,RU,42.8190,47.1192,89,Мика-центр
569,ikwiriri,TZ,-7.9562,38.9716,93,No hotel found
572,trairi,BR,-3.2778,-39.2689,82,Pousada Villa Aurora


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:


# Configure the map plot
hotels_country = hotel_df.hvplot.points(
    "Lng", "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
hotels_country



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)